This is a local, non-Kaggle notebook in which TFX 1.16.0 and python 3.10 and the compatible versions of other libraries are installed in a virtual environment that this notebook is running in.

paths are relative to the github repository directory, "recommender_systems"

In [1]:
import shutil

from tfx.orchestration import metadata

import tensorflow_transform as tft

from ml_metadata.proto import metadata_store_pb2
from ml_metadata.metadata_store import metadata_store

import sys
import os
sys.path.append(os.path.join(os.getcwd(), "src/test/python/movie_lens_tfx"))
sys.path.append(os.path.join(os.getcwd(), "src/main/python/movie_lens_tfx"))

from helper import *
from movie_lens_tfx.PipelineComponentsFactory import *
from movie_lens_tfx.tune_train_movie_lens import *

from absl import logging
tf.get_logger().propagate = False
logging.set_verbosity(logging.WARNING)
logging.set_stderrthreshold(logging.WARNING)

2025-11-04 16:56:04.918367: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 16:56:04.921382: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 16:56:04.964395: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-04 16:56:05.827259: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:absl:tensorflow_io is not available: No module named 'tensorflow_io'
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is

## EDA on the raw data

### w/ Polars and Plotly express
output is written to bin/local_notebook/images/

In [2]:
%run src/main/python/eda/eda_raw.py

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
key=ratings, file_path=/home/nichole/projects/github/recommender_systems/src/main/resources/ml-1m/ratings.dat
key=users, file_path=/home/nichole/projects/github/recommender_systems/src/main/resources/ml-1m/users.dat
key=movies, file_path=/home/nichole/projects/github/recommender_systems/src/main/resources/ml-1m/movies.dat
177 movies were not rated
0 users did not rate

ratings: DESCRIBE
<bound method DataFrame.describe of shape: (1_000_209, 4)
┌─────────┬──────────┬────────┬───────────┐
│ user_id ┆ movie_id ┆ rating ┆ timestamp │
│ ---     ┆ ---      ┆ ---    ┆ ---       │
│ i64     ┆ i64      ┆ i64    ┆ i64       │
╞═════════╪══════════╪════════╪═══════════╡
│ 1       ┆ 1193     ┆ 5      ┆ 978300760 │
│ 1       ┆ 661      ┆ 3      ┆ 978302109 │
│ 1       ┆ 914      ┆ 3      ┆ 978301968 │
│ 1       ┆ 3408     ┆ 4      ┆ 978300275 │
│ 1       ┆ 2355     ┆ 5      ┆ 978824291 │
│ …       ┆ …        ┆ …      ┆ …         │


### Run data pre-processing on full dataset to get the transformed data

In [3]:
infiles_dict_ser, output_config_ser, split_names = get_test_data(use_small=False)
user_id_max = 6040
movie_id_max = 3952
n_genres = N_GENRES
n_age_groups = N_AGE_GROUPS
n_occupations = 21
MIN_EVAL_SIZE = 50 #make this larger for production pipeline

test_num = "1"
    
PIPELINE_NAME = 'TestPipelines'
output_data_dir = os.path.join(get_bin_dir(), "local_notebook", test_num)
PIPELINE_ROOT = os.path.join(output_data_dir, PIPELINE_NAME)

# remove results from previous test runs:
try:
  print(f"removing: {PIPELINE_ROOT}")
  shutil.rmtree(PIPELINE_ROOT)
except OSError as e:
  pass
METADATA_PATH = os.path.join(PIPELINE_ROOT, 'tfx_metadata',
                             'metadata.db')
os.makedirs(os.path.join(PIPELINE_ROOT, 'tfx_metadata'),
            exist_ok=True)

ENABLE_CACHE = True

# metadata_connection_config = metadata_store_pb2.ConnectionConfig()
# metadata_connection_config.sqlite.SetInParent()
# metadata_connection = metadata.Metadata(metadata_connection_config)
metadata_connection_config = metadata.sqlite_metadata_connection_config(
  METADATA_PATH)

store = metadata_store.MetadataStore(metadata_connection_config)

if get_kaggle():
  tr_dir = "/kaggle/working/"
else:
  tr_dir = os.path.join(get_project_dir(), "src/main/python/movie_lens_tfx")

serving_model_dir = os.path.join(PIPELINE_ROOT, 'serving_model')
output_parquet_path = os.path.join(PIPELINE_ROOT, "transformed_parquet")

# for the custom ingestion component, the apache beam pipeline needs to be able to
# find the sibling scripts it imports.
# 2 solutions: (1) create a tar archive and use --extra_package in pipeline args
# or (2) use setup.py and --setup_file in pipeline args.

beam_pipeline_args = [
  '--direct_running_mode=multi_processing',
  '--direct_num_workers=0',
  '--setup_file=setup.py',
  #f'--extra_package={ingest_tar_file}'
]

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
removing: /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines


In [4]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

tf.get_logger().propagate = False
import logging
logging.getLogger().setLevel(logging.WARNING)
from absl import logging
logging.set_verbosity(logging.WARNING)
logging.set_stderrthreshold(logging.WARNING)

context = InteractiveContext(pipeline_name=PIPELINE_NAME, pipeline_root=PIPELINE_ROOT,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args
)

factory = PipelineComponentsFactory(infiles_dict_ser, output_config_ser, tr_dir,
    user_id_max, movie_id_max, n_genres, n_age_groups,
    MIN_EVAL_SIZE, serving_model_dir, output_parquet_path)

components = factory.build_components(PIPELINE_TYPE.PREPROCESSING)

for component in components:
    context.run(component)

print(f'done pre-processing data')

2025-11-04 16:57:07.366108: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 16:57:07.371490: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 16:57:07.373043: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 16:57:07.379080: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 16:57:07.381081: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 16:57:07.391436: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 16:57:07.434592: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not b

running bdist_wheel
running build
running build_py
creating build/lib
copying transform_movie_lens.py -> build/lib
copying PipelineComponentsFactory.py -> build/lib
copying tune_train_movie_lens.py -> build/lib
installing to /tmp/tmp59ls6_2c
running install
running install_lib
copying build/lib/transform_movie_lens.py -> /tmp/tmp59ls6_2c/.
copying build/lib/PipelineComponentsFactory.py -> /tmp/tmp59ls6_2c/.
copying build/lib/tune_train_movie_lens.py -> /tmp/tmp59ls6_2c/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Trans

/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:90: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+cd2e3e10c521d87ebc928be4ec699c775b2b3bf2739e81beb77dce963548d40e-py3-none-any.whl


INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'transform_movie_lens@/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+cd2e3e10c521d87ebc928be4ec699c775b2b3bf2739e81beb77dce963548d40e-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+cd2e3e10c521d87ebc928be4ec699c775b2b3bf2739e81beb77dce963548d40e-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/home/nichole/miniconda3/envs/tfx_py310/bin/python3.10', '-m', 'pip', 'install', '--target', '/tmp/tmpc8a25yeq', '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+cd2e3e10c521d87ebc928be4ec699c775b2b3bf2739e81beb77dce963548d40e-py3-none-any.whl']


...

INFO:absl:Successfully installed '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+cd2e3e10c521d87ebc928be4ec699c775b2b3bf2739e81beb77dce963548d40e-py3-none-any.whl'.
INFO:absl:Installing '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+cd2e3e10c521d87ebc928be4ec699c775b2b3bf2739e81beb77dce963548d40e-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/home/nichole/miniconda3/envs/tfx_py310/bin/python3.10', '-m', 'pip', 'install', '--target', '/tmp/tmpchei5sl4', '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+cd2e3e10c521d87ebc928be4ec699c775b2b3bf2739e81beb77dce963548d40e-py3-none-any.whl']


Processing ./bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+cd2e3e10c521d87ebc928be4ec699c775b2b3bf2739e81beb77dce963548d40e-py3-none-any.whl


INFO:absl:Successfully installed '/home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines/_wheels/tfx_user_code_transform-0.0+cd2e3e10c521d87ebc928be4ec699c775b2b3bf2739e81beb77dce963548d40e-py3-none-any.whl'.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature genres has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occupation has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature rating has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature timestamp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.


INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature genres has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occupation has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature rating has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature timestamp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature user_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature genres has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature movie_id has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature occu

done pre-processing data


## EDA on the transformed data
output is written to bin/local_notebook/images/

### using Polars, Plotly.express 

this can take an hour on a single COTS

In [5]:
%run src/main/python/eda/eda_transformed.py

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
CWD=/home/nichole/projects/github/recommender_systems, kaggle=False

train df: shape: (5, 14)
┌───────┬─────┬────────┬───────────────────┬──────┬───────┬───────┬──────────┬────────────┬────────┬─────────────┬─────────┬─────────┬──────┐
│ row   ┆ age ┆ gender ┆ genres            ┆ hr   ┆ hr_wk ┆ month ┆ movie_id ┆ occupation ┆ rating ┆ sec_into_yr ┆ user_id ┆ weekday ┆ yr   │
│ ---   ┆ --- ┆ ---    ┆ ---               ┆ ---  ┆ ---   ┆ ---   ┆ ---      ┆ ---        ┆ ---    ┆ ---         ┆ ---     ┆ ---     ┆ ---  │
│ u32   ┆ f32 ┆ f32    ┆ list[f32]         ┆ f32  ┆ f32   ┆ f32   ┆ i32      ┆ f32        ┆ i64    ┆ i32         ┆ f32     ┆ f32     ┆ i32  │
╞═══════╪═════╪════════╪═══════════════════╪══════╪═══════╪═══════╪══════════╪════════════╪════════╪═════════════╪═════════╪═════════╪══════╡
│ 39363 ┆ 0.0 ┆ 0.0    ┆ [0.0, 0.0, … 0.0] ┆ 17.0 ┆ 126.0 ┆ 377.0 ┆ 938      ┆ 10.0       ┆ 4      ┆ 31512552    ┆ 1.0     ┆ 7.0

### using Pandas, Pyspark MLLIB FPGrowth

This does a market basket analysis with movie_ids.

If you want the PrefixSpan plots also, set
PLOT_PREFIXSPAN = True
but beware that the script will take much longer to run.

In [6]:
PLOT_PREFIXSPAN=False

In [7]:
%run src/main/python/eda/eda_transformed_pyspark_mllib.py

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
CWD=/home/nichole/projects/github/recommender_systems, kaggle=False

df w/ occ: shape: (5, 15)
┌───────┬─────┬────────┬───────────────────┬──────┬───────┬───────┬──────────┬────────────┬────────┬─────────────┬─────────┬─────────┬──────┬──────────────┐
│ row   ┆ age ┆ gender ┆ genres            ┆ hr   ┆ hr_wk ┆ month ┆ movie_id ┆ occupation ┆ rating ┆ sec_into_yr ┆ user_id ┆ weekday ┆ yr   ┆ occ          │
│ ---   ┆ --- ┆ ---    ┆ ---               ┆ ---  ┆ ---   ┆ ---   ┆ ---      ┆ ---        ┆ ---    ┆ ---         ┆ ---     ┆ ---     ┆ ---  ┆ ---          │
│ u32   ┆ f32 ┆ f32    ┆ list[f32]         ┆ f32  ┆ f32   ┆ f32   ┆ i32      ┆ f32        ┆ i64    ┆ i32         ┆ f32     ┆ f32     ┆ i32  ┆ str          │
╞═══════╪═════╪════════╪═══════════════════╪══════╪═══════╪═══════╪══════════╪════════════╪════════╪═════════════╪═════════╪═════════╪══════╪══════════════╡
│ 39363 ┆ 0.0 ┆ 0.0    ┆ [0.0, 0.0, … 0.0] ┆ 17.0 ┆ 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/11/04 17:32:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
dtype=[List(Int32)]
head=shape: (5, 1)
┌──────────────────────┐
│ movie_ids            │
│ ---                  │
│ list[i32]            │
╞══════════════════════╡
│ [318, 527, … 3869]   │
│ [3408]               │
│ [1093, 1210, … 3623] │
│ [111, 235, … 3809]   │
│ [6, 58, … 3535]      │
└──────────────────────┘
pandas_df.head=                                           movie_ids
0  [318, 527, 593, 903, 1193, 1230, 1645, 1964, 2...
1                                             [3408]
2  [1093, 1210, 2712, 2858, 2918, 2959, 2997, 340...
3  [111, 235, 260, 296, 318, 527, 593, 608, 858, ...
4  [6, 58, 147, 194, 260, 265, 281, 293, 296, 321...
pandas_df.dtypes=movie_ids    object
dtype: object


/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



spark_df schema=
root
 |-- movie_ids: array (nullable = true)
 |    |-- element: integer (containsNull = false)



...

head: [Row(movie_ids=[1967, 3287, 3394, 3566]), Row(movie_ids=[540, 771, 916, 971, 1227, 2361])]
construct FPGrowth model
TYPE=<class 'pyspark.sql.dataframe.DataFrame'>

frequent_itemsets_spark:
+------+----+
| items|freq|
+------+----+
|[3809]|  33|
|[1208]|  26|
|[2605]|  28|
| [588]|  30|
|[1036]|  24|
|[2688]|  32|
| [339]|  28|
|[2858]|  42|
|[1909]|  36|
|[2710]|  30|
|[2916]|  57|
| [223]|  40|
|[1394]|  35|
|[1376]|  31|
|[1374]|  38|
|[3638]|  24|
|[2291]|  46|
| [653]|  27|
|[3101]|  25|
|[2243]|  25|
+------+----+
only showing top 20 rows


filtered_itemsets:
+-----+----+
|items|freq|
+-----+----+
+-----+----+

results_pd len=0
sorted by support=Empty DataFrame
Columns: [items, freq, itemset_str, support]
Index: []
filtered.columns=['row', 'age', 'gender', 'genres', 'hr', 'hr_wk', 'month', 'movie_id', 'occupation', 'rating', 'sec_into_yr', 'user_id', 'weekday', 'yr', 'occ']
filtered head=shape: (2, 15)
┌──────┬─────┬────────┬──────────┬──────┬───────┬───────┬──────────┬─────

/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



head: [Row(movie_ids=[185, 1204, 1722, 2709, 3418, 3802]), Row(movie_ids=[888, 1517, 1983, 2107, 2169, 2392, 2412, 2568, 3300])]
construct FPGrowth model
TYPE=<class 'pyspark.sql.dataframe.DataFrame'>

frequent_itemsets_spark:
+------+----+
| items|freq|
+------+----+
|[2792]|  18|
| [788]|  22|
|[2683]|  18|
|[3697]|  17|
|  [95]|  20|
| [377]|  18|
|[2628]|  27|
|[2916]|  20|
|[2805]|  17|
| [442]|  19|
|[2995]|  17|
|[3033]|  18|
| [292]|  17|
|[1562]|  22|
|[1320]|  19|
|[1544]|  36|
|[2422]|  18|
|[2657]|  17|
| [185]|  20|
|[3175]|  19|
+------+----+
only showing top 20 rows


filtered_itemsets:
+-----+----+
|items|freq|
+-----+----+
+-----+----+

results_pd len=0
sorted by support=Empty DataFrame
Columns: [items, freq, itemset_str, support]
Index: []
filtered.columns=['row', 'age', 'gender', 'genres', 'hr', 'hr_wk', 'month', 'movie_id', 'occupation', 'rating', 'sec_into_yr', 'user_id', 'weekday', 'yr', 'occ']
filtered head=shape: (2, 15)
┌───────┬─────┬────────┬────────┬──────┬─

/home/nichole/miniconda3/envs/tfx_py310/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



head: [Row(movie_ids=[2520]), Row(movie_ids=[2169, 3354])]
construct FPGrowth model
TYPE=<class 'pyspark.sql.dataframe.DataFrame'>

frequent_itemsets_spark:
+------+----+
| items|freq|
+------+----+
| [160]|  13|
|[2628]|  19|
| [231]|  13|
|[1544]|  11|
| [849]|  15|
|[1391]|  11|
|[1573]|  13|
|[2710]|  24|
|[3273]|  16|
|[1976]|  11|
|[2412]|  14|
|[3802]|  11|
|[1676]|  13|
|[2720]|  17|
|[3257]|  12|
|[2986]|  15|
| [546]|  28|
|[2422]|  13|
|[1100]|  11|
|[1917]|  16|
+------+----+
only showing top 20 rows


filtered_itemsets:
+-----+----+
|items|freq|
+-----+----+
+-----+----+

results_pd len=0
sorted by support=Empty DataFrame
Columns: [items, freq, itemset_str, support]
Index: []
wrote multivariate EDA images to /home/nichole/projects/github/recommender_systems/bin/local_notebook/images


### Data and Concept Drift
After exploring the data inputs to the model, we want to define monitoring
for data and concept shifts.

let X = features

let Y = targets

Data shift is a change in the joint distribution, P(X, Y). 

Using the probability product rule, we can explore 4 causes for the
simplest changes in P(X,Y)
$$ P(X, Y) = P(Y, X) = P(X|Y)P(Y) = P(Y|X)P(X) $$

We can look for changes in one member in the following pairs at any time
(one is simpler than exploring more than 1 member changing at same time):
$$ P(X|Y) * P(Y) $$
$$ P(Y|X) * P(X) $$

* Covariate shift:
  $$ P(X) changed.  P(Y|X) unchanged $$
  Distr of model inputs changes.
* Label shift:
  $$ P(Y) changed,  P(X|Y) unchanged $$
  Distr of model outputs changes, but for any given output, the input distribution stays the same.
  
* Concept shift:
  $$ P(X) unchanged,  P(Y|X) changed $$
* Manifestation shift:
  $$ P(Y) unchanged,  P(X|Y) changed $$

[see more at NannyML](https://www.nannyml.com/blog/concept-drift)

if the train, eval, and test split are random, then one could use
the maximum differences in their distributions as a lower limit on 
the estimate for stochastic error.  A trigger for data drift should
then be GEQ about 3 times that stochastic error.

The previous data doesn't exist yet for this project, but one could
either download another movie-lens dataset of different time period,
or split this 1M dataset by a timestamp ordering into 2 partitions.

### using TFDV to look at the raw data

In [23]:
from tfx.dsl.io import fileio
from tfx.orchestration import metadata
from tfx.components import StatisticsGen, SchemaGen, ExampleValidator
from tfx.utils import io_utils
from tensorflow_metadata.proto.v0 import anomalies_pb2, schema_pb2
from tensorflow_metadata.proto.v0 import statistics_pb2
from tensorflow_transform.tf_metadata import schema_utils
import tensorflow_data_validation as tfdv

_list = store.get_artifacts_by_type("ExampleStatistics")
print(f'ExampleStatistics count={len(_list)}')
#print(f'ExampleStatistics ={_list}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "StatisticsGen" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "FeatureStats.pb") 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    try:
        with open(file_path, 'rb') as f:
            serialized_stats = f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")    
    stats = statistics_pb2.DatasetFeatureStatisticsList()
    stats.ParseFromString(serialized_stats)
    tfdv.visualize_statistics(stats)

#ExampleAnomalies
_list = store.get_artifacts_by_type("ExampleAnomalies")
print(f'ExampleAnomalies count={len(_list)}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "ExampleValidator" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "SchemaDiff.pb") 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    try:
        with open(file_path, 'rb') as f:
            serialized_anomalies = f.read()
    except FileNotFoundError:
        print(f"Error: Anomaly file not found at {file_path}")
    anomalies = anomalies_pb2.Anomalies()
    anomalies.ParseFromString(serialized_anomalies)
    tfdv.display_anomalies(anomalies)

_list = store.get_artifacts_by_type("ExampleStatistics")
#print(f'ExampleStatistics={_list}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "pre_transform_stats" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name) 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    try:
        with open(file_path, 'rb') as f:
            serialized_stats = f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")    
    stats = statistics_pb2.DatasetFeatureStatisticsList()
    stats.ParseFromString(serialized_stats)
    tfdv.visualize_statistics(stats)
    

ExampleStatistics count=3


ExampleAnomalies count=2


### using TFDV to look at the transformed data

In [25]:

_list = store.get_artifacts_by_type("ExampleStatistics")
#print(f'artifacts={_list}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "post_transform_stats" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name) 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    try:
        with open(file_path, 'rb') as f:
            serialized_stats = f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")    
    stats = statistics_pb2.DatasetFeatureStatisticsList()
    stats.ParseFromString(serialized_stats)
    tfdv.visualize_statistics(stats)
    
#ExampleAnomalies
_list = store.get_artifacts_by_type("ExampleAnomalies")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "post_transform_anomalies" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name) 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    anomalies = anomalies_pb2.Anomalies()
    try:
        with open(file_path, 'rb') as f:
            serialized_anomalies = f.read()
    except FileNotFoundError:
        print(f"Error: Anomaly file not found at {file_path}")
    anomalies = anomalies_pb2.Anomalies()
    anomalies.ParseFromString(serialized_anomalies)
    tfdv.display_anomalies(anomalies)


## Run baseline model pipeline with full dataset

pipeline_factory = PipelineComponentsFactory(
  infiles_dict_ser=infiles_dict_ser, output_config_ser=output_config_ser,
  transform_dir=tr_dir, user_id_max=user_id_max, movie_id_max=movie_id_max,
  n_genres=n_genres, n_age_groups=n_age_groups, min_eval_size=MIN_EVAL_SIZE,
  serving_model_dir=serving_model_dir,
)

beam_pipeline_args = [
  '--direct_running_mode=multi_processing',
  '--direct_num_workers=0'
    ]

baseline_components = pipeline_factory.build_components(MODEL_TYPE.BASELINE)
    
# create baseline model
my_pipeline = tfx.dsl.Pipeline(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  components=baseline_components,
  enable_cache=ENABLE_CACHE,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args,
)

tfx.orchestration.LocalDagRunner().run(my_pipeline)

artifact_types = store.get_artifact_types()
logging.debug(f"MLMD store artifact_types={artifact_types}")
artifacts = store.get_artifacts()
logging.debug(f"MLMD store artifacts={artifacts}")

components = pipeline_factory.build_components(MODEL_TYPE.PRODUCTION)
# simulate experimentation of one model family
my_pipeline = tfx.dsl.Pipeline(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  components=components,
  enable_cache=ENABLE_CACHE,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args,
)

tfx.orchestration.LocalDagRunner().run(my_pipeline)


artifact_types = store.get_artifact_types()
print(f"MLMD store artifact_types={artifact_types}")
artifacts = store.get_artifacts()
print(f"MLMD store artifacts={artifacts}")

executions = store.get_executions()
logging.debug(f"MLMD store executions={executions}")

# executions has custom_properties.key: "infiles_dict_ser"
#    and custom_properties.key: "output_config_ser"
artifact_count = len(artifacts)
execution_count = len(executions)
